In [ ]:
using Random, LinearAlgebra, StatsBase

function acor_optimize(;
    f,                # Целевая функция
    dims,             # Размерность задачи
    ants_num = 50,    # Количество муравьев
    iterations = 100, # Число итераций
    archive_size = 50,# Размер архива решений
    q = 0.5,          # Параметр интенсивности
    ξ = 0.85,         # Коэффициент испарения
    bounds = nothing  # Границы поиска [optional]
)
    if bounds === nothing
        archive = rand(dims, archive_size)
    else
        archive = [bounds[i][1] + rand()*(bounds[i][2]-bounds[i][1]) for i in 1:dims, _ in 1:archive_size]
    end
    
    f_values = [f(archive[:,i]) for i in 1:archive_size]
    
    for iter in 1:iterations

        order = sortperm(f_values)
        sorted_archive = archive[:, order]
        sorted_f = f_values[order]
        
        weights = [exp(-(i-1)/(q*archive_size)) for i in 1:archive_size]
        weights ./= sum(weights)
        
        new_solutions = zeros(dims, ants_num)
        for k in 1:ants_num

            i = sample(1:archive_size, Weights(weights))

            σ = ξ * sum(abs.(sorted_archive .- sorted_archive[:,i]), dims=2) ./ (archive_size-1)
            
            new_solution = sorted_archive[:,i] .+ σ .* randn(dims)
            
            if bounds !== nothing
                new_solution = clamp.(new_solution, [b[1] for b in bounds], [b[2] for b in bounds])
            end
            
            new_solutions[:,k] = new_solution
        end
        
        new_f = [f(new_solutions[:,k]) for k in 1:ants_num]
        
        combined_archive = hcat(archive, new_solutions)
        combined_f = vcat(f_values, new_f)
        
        best_indices = partialsortperm(combined_f, 1:archive_size)
        archive = combined_archive[:, best_indices]
        f_values = combined_f[best_indices]
        
        if iter % 10 == 0
            println("Итерация $iter, лучший результат: $(minimum(f_values))")
        end
    end
    
    best_idx = argmin(f_values)
    return archive[:,best_idx], f_values[best_idx]
end

rosenbrock(x) = (1.0 - x[1])^2 + 100.0*(x[2] - x[1]^2)^2

best_solution, best_value = acor_optimize(
    f = rosenbrock,
    dims = 2,
    ants_num = 100,
    iterations = 200,
    archive_size = 50,
    bounds = [(-5.0, 5.0), (-5.0, 5.0)]
)

println("\nЛучшее решение: ", best_solution)
println("Минимальное значение: ", best_value)

Итерация 10, лучший результат: 0.056099810806081715
Итерация 20, лучший результат: 0.0008649106642500012
Итерация 30, лучший результат: 0.0008649106642500012
Итерация 40, лучший результат: 0.00015440969689984487
Итерация 50, лучший результат: 7.263378975702162e-7
Итерация 60, лучший результат: 7.263378975702162e-7
Итерация 70, лучший результат: 7.263378975702162e-7
Итерация 80, лучший результат: 4.133780114396674e-7
Итерация 90, лучший результат: 8.098553082182158e-8
Итерация 100, лучший результат: 5.0697653930404435e-8
Итерация 110, лучший результат: 3.817858810789065e-11
Итерация 120, лучший результат: 3.817858810789065e-11
Итерация 130, лучший результат: 3.549502869895033e-11
Итерация 140, лучший результат: 6.556503621179906e-12
Итерация 150, лучший результат: 1.6044002571494962e-12
Итерация 160, лучший результат: 1.7974241393206818e-13
Итерация 170, лучший результат: 5.693355279218404e-14
Итерация 180, лучший результат: 3.151567779299681e-15
Итерация 190, лучший результат: 1.002005